In [1]:
import pandas as pd
from src.gru.gru import gru
from src.parser.parse import Parser
from src.gru.gru import GRUModel
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from src.wiener_kolmogorov.wiener_kolmogorov import WienerKolmogorov as WK
from src.detector.detector import  Detector
import numpy as np
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from keras.models import load_model

2022-05-23 19:47:35.634224: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-23 19:47:35.634270: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def rates(true, pre, positives, negatives):
    tn, fp, fn, tp = confusion_matrix(true, pre).ravel()
    return accuracy_score(true, pre), tp / positives, tn / negatives

In [3]:
def lstm_data_transform(x_data, y_data, num_steps=5):
    X, y = list(), list()
    for i in range(x_data.shape[0]):
        end_ix = i + num_steps
        if end_ix >= x_data.shape[0]:
            break
        seq_X = x_data[i:end_ix]
        seq_y = y_data[end_ix]
        X.append(seq_X)
        y.append(seq_y)
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

In [4]:
def clean(preds):
    res= []
    for pred in preds:
        d0 = abs(pred - 0)
        d1 = abs(pred - 1)
        if d0 < d1:
            res.append(0)
        else:
            res.append(1)
    return res

In [5]:

df = pd.read_csv('../../runs/3/office.csv')
df = df[:2100]
df = df[["RSSI"]]

window = 300
zeros = np.zeros(window)
ones = np.ones(window)
y = ones
l = len(ones) + window
f = True
while l <= 2100:
    if f:
        y = np.concatenate((y, zeros))
        f = False
    else:
        y = np.concatenate((y, ones))
        f = True
    l += window
num_steps = 500




In [7]:
df.shape

(2100, 1)

In [9]:
X.shape

(1600, 500)

In [6]:
X, y = lstm_data_transform(df["RSSI"].to_numpy(), y, num_steps=num_steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.285)

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, GRU, SimpleRNN, Activation
from keras.layers import Dropout
from keras.regularizers import l2
import numpy as np
from keras import optimizers

In [ ]:


model = Sequential()
model.add(GRU(200, activation='tanh', input_shape=(num_steps, 1),
               return_sequences=False))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=1, activation='linear'))
model.compile(optimizer="adam", loss='mse')

In [6]:
model.fit(X_train, y_train, epochs=35)

Epoch 1/35
36/36 [==============================] - 15s 374ms/step - loss: 0.2574
Epoch 2/35
36/36 [==============================] - 13s 367ms/step - loss: 0.1563
Epoch 3/35
36/36 [==============================] - 14s 378ms/step - loss: 0.1173
Epoch 4/35
36/36 [==============================] - 13s 367ms/step - loss: 0.1113
Epoch 5/35
36/36 [==============================] - 13s 366ms/step - loss: 0.1127
Epoch 6/35
36/36 [==============================] - 13s 370ms/step - loss: 0.1159
Epoch 7/35
36/36 [==============================] - 13s 369ms/step - loss: 0.0899
Epoch 8/35
36/36 [==============================] - 13s 365ms/step - loss: 0.1033
Epoch 9/35
36/36 [==============================] - 13s 367ms/step - loss: 0.1062
Epoch 10/35
36/36 [==============================] - 13s 368ms/step - loss: 0.0801
Epoch 11/35
36/36 [==============================] - 13s 366ms/step - loss: 0.0642
Epoch 12/35
36/36 [==============================] - 14s 379ms/step - loss: 0.0545
Epoch 13/35
3

In [27]:
model.fit(X_train, y_train, epochs=1)

36/36 [==============================] - 15s 414ms/step - loss: 0.0367


In [3]:
saved = load_model("./models/office1")

2022-05-23 19:48:12.881086: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-23 19:48:12.881137: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-23 19:48:12.881168: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (paavo-Swift-SF314-42): /proc/driver/nvidia/version does not exist
2022-05-23 19:48:12.882640: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [30]:
cleaned = clean(saved.predict(X_test))
accuracy_score(y_test, cleaned)

0.9802631578947368

In [21]:
saved.fit(X_train, y_train, epochs=1)

36/36 [==============================] - 15s 374ms/step - loss: 0.0519


In [10]:
saved.layers[1].input

<KerasTensor: shape=(None, 200) dtype=float32 (created by layer 'gru')>

In [24]:
model.fit(X_train, y_train, epochs=1)

36/36 [==============================] - 14s 379ms/step - loss: 0.0455


In [30]:
cleaned = clean(model.predict(X_test))
accuracy_score(y_test, cleaned)

0.9912280701754386

In [18]:
model.save("./models/office1")

INFO:tensorflow:Assets written to: ./models/office1/assets


INFO:tensorflow:Assets written to: ./models/office1/assets


In [31]:
cleaned = clean(saved.predict(X_test))
acc, tpr, tnr = rates(y_test, cleaned, sum(cleaned), len(cleaned) - sum(cleaned))
print(acc, tpr, tnr)

0.9802631578947368 0.9961832061068703 0.9587628865979382


In [7]:
cleaned = clean(saved.predict(X_test))
acc, tpr, tnr = rates(y_test, cleaned, sum(y_test), len(cleaned) - sum(y_test))
print(acc, tpr, tnr)

0.9736842105263158 0.97265625 0.975
